<a href="https://colab.research.google.com/github/MichaelWoo-git/Plant_Pathology_Apple_Trees/blob/main/plant_pathology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import

In [51]:
import numpy as np
import pandas as pd

Mount Google Drive data files

In [52]:
# #Just do a manual mounting using the button
# from google.colab import drive
# drive.mount('/gdrive')
# #This is if you want to just have file explorer to focus on the single folder
# # !ln -s "/gdrive/My Drive/data_files_kaggle" "/content/data"

Read in data

In [53]:
train_data = pd.read_csv("drive/MyDrive/data_files_kaggle/plant_pathology/train.csv")

In [54]:
train_data.head()

,image,labels
0,800113bb65efe69e.jpg,healthy
1,8002cb321f8bfcdf.jpg,scab frog_eye_leaf_spot complex
2,80070f7fb5e2ccaa.jpg,scab
3,80077517781fb94f.jpg,scab
4,800cbf0ff87721f8.jpg,complex


We are going to have to make directorys into the labels that are given so that when we use keras imagedatagenerator it will be a lot easier!

In [55]:
# import os
# from shutil import copyfile
# for file in os.listdir("drive/MyDrive/data_files_kaggle/plant_pathology/train_images"):
#   #print(file)
#   temp = train_data[train_data["image"] == file]
#   label = np.array(temp["labels"])[0]
#   print("drive/MyDrive/data_files_kaggle/plant_pathology/train_data/"+label)
#   if not os.path.exists("drive/MyDrive/data_files_kaggle/plant_pathology/train_data/"+label):
#     os.makedirs("drive/MyDrive/data_files_kaggle/plant_pathology/train_data/"+label)
#   if not os.path.exists("drive/MyDrive/data_files_kaggle/plant_pathology/train_data/"+label + "/"+file):
#     copyfile("drive/MyDrive/data_files_kaggle/plant_pathology/train_images/"+file,"drive/MyDrive/data_files_kaggle/plant_pathology/train_data/"+label + "/"+file)

We are going to generate data from these images from the train_data directory

In [56]:
from keras_preprocessing.image import ImageDataGenerator
train_path = "drive/MyDrive/data_files_kaggle/plant_pathology/train_data"
train_datagen = ImageDataGenerator().flow_from_directory(train_path,batch_size=100)

Found 18632 images belonging to 12 classes.


train data and labels

In [57]:
x,y = next(train_datagen)
#formatting output
y = np.array(y)
y = np.array([np.argmax(p) for p in y])
#formatting input
x = np.array(x)
#we have to reshape the data into 2 dimmensions so the classifers can read in the data
x = x.reshape((-1,32*32*32*3*2))

Generate the model (Random Forest)

In [58]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)



In [59]:
print(y_pred)

[1 3 1 1 3 9 3 1 3 1 1 3 3 1 9 3 1 3 3 3]


In [60]:
print(y_test)

[ 1  1  3  5  1  4  9  1  4  3  9  9 10  3  6  1  1  9  9  9]


Confusion Matrix

In [61]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,y_pred))

[[3 3 0 0 0 0 0]
 [3 0 0 0 0 0 0]
 [0 1 0 0 0 1 0]
 [1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0]
 [1 5 0 0 0 0 0]
 [0 1 0 0 0 0 0]]


In [62]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.38      0.50      0.43         6
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         6
          10       0.00      0.00      0.00         1

    accuracy                           0.15        20
   macro avg       0.05      0.07      0.06        20
weighted avg       0.11      0.15      0.13        20



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
